# Find biological motifs

To make sure our sampling strategy makes sense, we manually enter a few biological motifs.
We then look for these motifs in a large random sample, as well as in a large biological random sample.
If everything is correct, we should find the real motifs more frequently in the latter sample than in the former.

In [ ]:
from discrete_motif import DiscreteGrnMotif
import discrete_motif_functions as functions
import discrete_motif_generator as generator
import discrete_motif_plotting as visualize
import discrete_motif_measures as measures
import discrete_motif_operations as operations
from IPython.display import HTML, display
import tabulate

from copy import deepcopy
import numpy as np
import os
import pickle
import random

In [ ]:
# define a few real motifs
real_motifs_dict = {}

# make a motif
motif = generator.generate_random(1, 2, 2)[0]
motif.transition_table = [[1, 1, 1, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 0, 1]]
motifs = []
motifs.append(motif)
real_motifs_dict["22"] = motifs

# make a motif
motif = generator.generate_random(1, 2, 3)[0]
motif.transition_table = [[2, 2, 2, 0], [2, 1, 2, 0], [2, 0, 2, 0], [1, 2, 1, 0], [1, 1, 1, 0], [1, 0, 1, 0], [0, 2, 0, 1], [0, 1, 0, 1], [0, 0, 0, 1]]
motifs = []
motifs.append(motif)
real_motifs_dict["23"] = motifs

# make a motif
motifs = []
motif = generator.generate_random(1, 3, 3)[0]
motif.transition_table = [[2, 2, 2, 2, 2, 0], [2, 2, 1, 2, 2, 0], [2, 2, 0, 2, 2, 0], [2, 1, 2, 2, 1, 0], [2, 1, 1, 2, 1, 0], [2, 1, 0, 2, 1, 0], [2, 0, 2, 2, 0, 0], [2, 0, 1, 2, 0, 0], [2, 0, 0, 2, 0, 0], [1, 2, 2, 1, 2, 0], [1, 2, 1, 1, 2, 0], [1, 2, 0, 1, 2, 0], [1, 1, 2, 1, 1, 0], [1, 1, 1, 1, 1, 0], [1, 1, 0, 1, 1, 0], [1, 0, 2, 1, 0, 1], [1, 0, 1, 1, 0, 1], [1, 0, 0, 1, 0, 1], [0, 2, 2, 0, 2, 0], [0, 2, 1, 0, 2, 0], [0, 2, 0, 0, 2, 0], [0, 1, 2, 0, 1, 0], [0, 1, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0], [0, 0, 2, 0, 0, 2], [0, 0, 1, 0, 0, 2], [0, 0, 0, 0, 0, 2]]
motifs.append(motif)
real_motifs_dict["33"] = motifs

In [ ]:
# generate some big samples to find these in    
sample_size = 1000
samples = []

# 22
network_size = 2
logic_size = 2
sample = []
sample.append(generator.generate_random(sample_size, network_size, logic_size))
sample.append(generator.generate_motifs(sample_size, network_size, logic_size)[0])
sample.append([real_motifs_dict["22"]])
samples.append(sample)

# 23
network_size = 2
logic_size = 3
sample = []
sample.append(generator.generate_random(sample_size, network_size, logic_size))
sample.append(generator.generate_motifs(sample_size, network_size, logic_size)[0])
sample.append([real_motifs_dict["23"]])
samples.append(sample)

# 33
network_size = 3
logic_size = 3
sample = []
sample.append(generator.generate_random(sample_size, network_size, logic_size))
sample.append(generator.generate_motifs(sample_size, network_size, logic_size)[0])
sample.append([real_motifs_dict["33"]])
samples.append(sample)

In [ ]:
for i in range(0, len(samples)):
    args = (samples[i][0][0].grn_vars["gene_cnt"], samples[i][0][0].numvalues)
    print("Using %s nodes, %s-valued logic" % args)
    sample_random = samples[i][0]
    sample_bio = samples[i][1]
    real_motifs = samples[i][2]

    bio_total = 0
    random_total = 0
    bio_matched_motifs = []
    random_matched_motifs = []
    
    for real_motif in real_motifs:
        real_motif = real_motif[0]
        random_matches = real_motif.find_in_sample(sample_random, strict=False)
        bio_matches = real_motif.find_in_sample(sample_bio, strict=False)

        if len(random_matches) > 0:
            random_total += len(random_matches)
            random_matched_motifs.append(real_motif)
        if len(bio_matches) > 0:
            bio_total += len(bio_matches)
            bio_matched_motifs.append(real_motif)

    args = (bio_total, len(bio_matched_motifs), len(real_motifs))
    print("Found %d matches in the bio sample, for a total of %d out of %d real motifs." % args)
    args = (random_total, len(random_matched_motifs), len(real_motifs))
    print("Found %d matches in the random sample, for a total of %d out of %d real motifs." % args)

In [ ]:
def loop_impacts(network_size, nudge_size, motif):
    # find the memory
    memory = measures.normalized_memory(motif)
    
    # try to find the synergy
    synergy = measures.normalized_synergy(motif, synergy_measure)
    
    # try to loop
    impacts = []
    for nudge_width in range(1, network_size + 1):
        if not (nudge_method == 'joint_pdf' and nudge_width == network_size):
            # we compare the two evolved states
            impact = measures.average_nudge_impact(motif, nudge_width, nudge_size, nudge_method)
            impact_tuple = (nudge_width, impact)
            impacts.append(impact_tuple)
    
    return (synergy, impacts, memory)

In [ ]:
synergy_measure = measures.synergy_middleground
nudge_method = "DJ"
for key, value in real_motifs_dict.iteritems():
    for motif in value:
        
        args = loop_impacts(motif.numvariables, 0.25, motif)
        
        print("Synergy: " + str(args[0]))
        print("Memory: " + str(args[2]))
        print("Impacts...")
        print(args[1])
        print("Transition table...")
        print(motif.transition_table)

# Finding cycles in our samples

In [ ]:
cycletest = DiscreteGrnMotif(1, 2, 'random')
cycletest.grn_vars["gene_cnt"] = 2
cycletest.grn_vars["conflict_rule"] = 'totaleffect'
cycletest.append_rule([0], [0], functions.minus)
cycletest.append_rule([1], [1], functions.minus)
cycletest.append_rule([0], [1], functions.plus)
cycletest.append_rule([1], [0], functions.plus)
cycletest.construct_grn()
cycletest.evaluate_motif(genes=[0, 1])
print("The states: ")
for state in cycletest.states:
    print(state)
table = visualize.state_transition_table(cycletest, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))
print(cycletest.transition_table)
print(cycletest.set_transition_table())
print(cycletest.transition_table)
print(cycletest.is_cyclical())

In [ ]:
cycletest = DiscreteGrnMotif(1, 2, 'random')
cycletest.grn_vars["gene_cnt"] = 3
cycletest.grn_vars["conflict_rule"] = 'totaleffect'
cycletest.append_rule([0], [0], functions.minus)
cycletest.append_rule([1], [1], functions.minus)
cycletest.append_rule([2], [2], functions.minus)
cycletest.append_rule([0], [1], functions.plus)
cycletest.append_rule([1], [2], functions.plus)
cycletest.append_rule([2], [0], functions.plus)
cycletest.construct_grn()
cycletest.evaluate_motif(genes=[0, 1, 2])
print("The states: ")
for state in cycletest.states:
    print(state)
table = visualize.state_transition_table(cycletest, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))
print(cycletest.transition_table)
print(cycletest.set_transition_table())
print(cycletest.transition_table)
print(cycletest.is_cyclical())

In [ ]:
for i in range(0, len(samples)):
    args = (samples[i][0][0].grn_vars["gene_cnt"], samples[i][0][0].numvalues)
    print("Using %s nodes, %s-valued logic" % args)
    random_cycles = []
    bio_cycles = []
    for sample in samples[i][0]:
        lengths = [len(x) for x in sample.is_cyclical()]
        if len(lengths) == 0:
            lengths = [0]
        avg_length = int(np.max(lengths))
        random_cycles.append(avg_length)
    for sample in samples[i][1]:
        lengths = [len(x) for x in sample.is_cyclical()]
        if len(lengths) == 0:
            lengths = [0]
        avg_length = int(np.max(lengths))
        bio_cycles.append(avg_length)
    
    colors = ["r", "b"]
    labels = ["random", "biological random"]
    title = "Distribution of cycle lengths (0 if no cycle is present)"
    axes_labels = ["Maximum length of cycle in motif", "Number of occurences"]
    visualize.plot_bar([random_cycles, bio_cycles], colors, labels, title, axes_labels=axes_labels)

# Examine a few motifs (draw)

In [ ]:
for i in range(0, len(samples)):
    args = (samples[i][0][0].grn_vars["gene_cnt"], samples[i][0][0].numvalues)
    print("Using %s nodes, %s-valued logic" % args)
    counter = 0
    for sample in samples[i][1]:
        print("\nExamining sample %d" % counter)
        print(sample.grn_vars["rules"])
        counter += 1
        if counter ==  3:
            break

# Examine a few initial matrices

In [ ]:
for i in range(0, len(samples)):
    args = (samples[i][0][0].grn_vars["gene_cnt"], samples[i][0][0].numvalues)
    print("Using %s nodes, %s-valued logic" % args)
    counter = 0
    for sample in samples[i][1]:
        print("\nExamining sample %d\n" % counter)
        print(sample.grn_vars["correlations"])
        print("\n")
        print(sample.states[0])
        counter += 1
        if counter == 1:
            break
    counter = 0
    for sample in samples[i][0]:
        print("\nExamining sample %d\n" % counter)
        print(sample.grn_vars["correlations"])
        print("\n")
        print(sample.states[0])
        counter += 1
        if counter == 1:
            break

# Known cases (transition tables)

Valideer measures op wat random motieven: synergy/and/copy.

In [ ]:
import discrete_motif_measures as measures
import discrete_motif_operations as operations


# XOR
motif = generator.generate_random(samplesize=1, no_nodes=2, numvalues=2)[0]
motif.transition_table = [[1, 1, 0, 0],[1, 0, 1, 1], [0, 1, 1, 1], [0, 0, 0, 0]]

# experiment: set to equal chances
motif.joint_probabilities.joint_probabilities = np.array([[0.25, 0.25], [0.25, 0.25]])

motif.evaluate_motif()
unnudged = motif.states[-1]
print("The states: ")
for state in motif.states:
    print(state)
print("The correlations: ")
print(motif.grn_vars["correlations"])
print("Transitions: ")
print(motif.transition_table)

print("The entropy: ")
print(motif.entropy())
print("The mutual information: ")
print(measures.mutual_information(motif))
print("The WMS information: ")
print(measures.synergy_wms(motif))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif))
print("The memory: ")
motif.reset_to_state(0)
print(measures.mi_decay(motif, 1))
print(measures.normalized_memory(motif))
motif.reset_to_state(0)
operations.nudge_variable(motif, [0], 0.25, 'DJ')
motif.evaluate_motif()
print("The nudge impact: ")
print(unnudged)
print(motif.states[-1])
print(measures.hellinger(unnudged, motif.states[-1]))
print(measures.average_nudge_impact(motif, 1, 0.25, 'DJ'))

visualize.plot_mi_profile([motif], "XOR", mode='maximum')

In [ ]:
# COPY
motif = generator.generate_random(samplesize=1, no_nodes=2, numvalues=2)[0]
motif.transition_table = [[1, 1, 1, 1],[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 0, 0]]

# experiment: set to equal chances
motif.joint_probabilities.joint_probabilities = np.array([[0.25, 0.25], [0.25, 0.25]])

motif.evaluate_motif()
unnudged = motif.states[-1]
print("The states: ")
for state in motif.states:
    print(state)
print("The correlations: ")
print(motif.grn_vars["correlations"])
print("Transitions: ")
print(motif.transition_table)

print("The entropy: ")
print(motif.entropy())
print("The mutual information: ")
print(measures.mutual_information(motif))
print("The WMS information: ")
print(measures.synergy_wms(motif))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif))
print("The memory: ")
motif.reset_to_state(0)
print(measures.mi_decay(motif, 1))
print(measures.normalized_memory(motif))
motif.reset_to_state(0)
operations.nudge_variable(motif, [0], 0.25, 'DJ')
motif.evaluate_motif()
print("The nudge impact: ")
print(unnudged)
print(motif.states[-1])
print(measures.hellinger(unnudged, motif.states[-1]))
print(measures.average_nudge_impact(motif, 1, 0.25, 'DJ'))

visualize.plot_mi_profile([motif], "COPY", mode='maximum')

In [ ]:
# AND
motif = generator.generate_random(samplesize=1, no_nodes=2, numvalues=2)[0]
motif.transition_table = [[1, 1, 1, 1],[1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 0]]

# experiment: set to equal chances
motif.joint_probabilities.joint_probabilities = np.array([[0.25, 0.25], [0.25, 0.25]])

motif.evaluate_motif()
unnudged = motif.states[-1]
print("The states: ")
for state in motif.states:
    print(state)
print("The correlations: ")
print(motif.grn_vars["correlations"])
print("Transitions: ")
print(motif.transition_table)

print("The entropy: ")
print(motif.entropy())
print("The mutual information: ")
print(measures.mutual_information(motif))
print("The WMS information: ")
print(measures.synergy_wms(motif))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif))
print("The memory: ")
motif.reset_to_state(0)
print(measures.mi_decay(motif, 1))
print(measures.normalized_memory(motif))
motif.reset_to_state(0)
operations.nudge_variable(motif, [0], 0.25, 'DJ')
motif.evaluate_motif()
print("The nudge impact: ")
print(unnudged)
print(motif.states[-1])
print(measures.hellinger(unnudged, motif.states[-1]))
print(measures.average_nudge_impact(motif, 1, 0.25, 'DJ'))

visualize.plot_mi_profile([motif], "AND", mode='maximum')